In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split

In [2]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', XGBRegressor())
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', SGDRegressor())
])

In [11]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithms=[xgb_pipe, sgd_pipe], algorithm_name=["Gradient boosted decision tree", "Linear regression"], online=[False, False])
    return thesis


In [12]:
thesis = full_run()

Finished experiment! Elapsed time: 0.521690845489502, total Elapsed time: 0.521690845489502, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.5356583595275879, total Elapsed time: 1.0573492050170898, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.5187070369720459, total Elapsed time: 1.5760562419891357, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.5114128589630127, total Elapsed time: 2.0874691009521484, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished e

In [13]:
results = thesis.resultsToDF()

In [14]:
results[["Algorithm","SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Algorithm","Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE  \
                                                                                      mean   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             3.555000   
                                                                Small             3.850000   
                                                 Half observed  Large            14.912500   
                                                                Small             6.245833   
                                                 Unobserved     Large            12.652500   
                                                                Small             5.828333   
                               No Drift          No Drift       No Drift          3.982500   
                               Sudden Drift      Fully observed Large             3.500000   
                                                                Small             3.785000   
                                                 Half observed  Large             4.765833   
                                                                Small             4.357500   
                                                 Unobserved     Large            16.534167   
                                                                Small             6.508333   
                               Sudden Shock      3 days         Large             4.155000   
                                                                Small             4.015000   
                                                 7 days         Large             4.220000   
                                                                Small             3.982500   
Linear regression              Incremental Drift Fully observed Large             5.060000   
                                                                Small             3.653333   
                                                 Half observed  Large            18.882500   
                                                                Small             6.267500   
                                                 Unobserved     Large            15.704167   
                                                                Small             5.703333   
                               No Drift          No Drift       No Drift          3.457500   
                               Sudden Drift      Fully observed Large             5.195000   
                                                                Small             3.613333   
                                                 Half observed  Large             7.402500   
                                                                Small             3.932500   
                                                 Unobserved     Large            19.900000   
                                                                Small             6.700000   
                               Sudden Shock      3 days         Large             3.625000   
                                                                Small             3.525000   
                                                 7 days         Large             3.607500   
                                                                Small             3.480000   

                                                                                           \
                                                                                      std   
Algorithm                      Drift type        Drift time     Drift magnitude             
Gradient boosted decision tree Incremental Drift Fully observed Large            0.284237   
                                                                Small            0.151598   
                                                 Half observed  Large            7.688757   
                                                

In [7]:
results.sort_values("SMAPE", ascending=False)   

,Algorithm,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
11,xgb,NaN,Sudden Drift,Large,Important,Unobserved,2031.405536,31.27
93,xgb,3.0,Sudden Drift,Large,Important,Unobserved,1992.940765,30.78
134,xgb,6.0,Sudden Drift,Large,Important,Unobserved,1968.763954,30.60
52,xgb,0.0,Sudden Drift,Large,Important,Unobserved,1948.163642,30.51
216,sgd,0.0,Sudden Drift,Large,Important,Unobserved,1777.514576,28.97
...,...,...,...,...,...,...,...,...
191,sgd,NaN,Incremental Drift,Large,Important,Fully observed,9.796169,1.59
273,sgd,3.0,Incremental Drift,Large,Important,Fully observed,9.805589,1.58
314,sgd,6.0,Incremental Drift,Large,Important,Fully observed,9.500200,1.57
173,sgd,NaN,Sudden Drift,Large,Important,Fully observed,9.687094,1.57
